In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql

In [ ]:
def db_connect():
    conn = pymysql.connect(
        host='fininsightdevdb.mysql.database.azure.com', 
        user='election@fininsightdevdb',
        port=3306,
        password='election',
        db='election',
        charset='utf8'
    )
    
    return conn

In [ ]:
def db_close(conn):
    conn.close()

In [ ]:
def get_urls():
    url = 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0'
    area_url = []
    
    res = requests.get(url).text
    soup = BeautifulSoup(res,'html.parser')
    crawls = soup.select('table[class="nowraplinks collapsible autocollapse navbox-inner"] tbody')[6]
    crawls = crawls.select('tr')[7]
    
    for crawl in crawls.select('li a'):
        area_url.append('https://ko.wikipedia.org'+crawl.get('href'))
    
    return area_url

In [ ]:
def get_area(area_url):
    stopword = ['선거통계', '선거구별 결과', '지역구', '비례대표']
    area_ls = []
    for url in area_url:
        res = requests.get(url).text
        soup = BeautifulSoup(res,'html.parser')
        crawl = soup.select('h3 span[class="mw-headline"]')
        for area in crawl:
            if area.text not in stopword:
                area_ls.append("('"+area.text+"')")
    
    return area_ls

In [ ]:
# conn = db_connect()
area_url = get_urls()
area_ls = get_area(area_url)
# db_close(conn)

In [ ]:
sql = "insert into area (aname) values "
ls = []
for area in area_ls:
    print(area)
